# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
torch.cuda.is_available()

True

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
from ddpg_multi_agent import Agent

def trainFunction(n_episodes=1000, num_agents = 20, init_eps = 1.0):
    agent = Agent(state_size=33, action_size=4, seed=37)
    eps = init_eps
    
    avg_scores = []
    scores_window = deque(maxlen=100)  # last 100 scores
    
    for i_episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        states = env_info.vector_observations
        scores = np.zeros(num_agents)
        while True:
            actions =[]
            for an_agent in range(num_agents):
                agent.reset()
                actions.append(agent.act(states[an_agent],eps))
            eps *=0.99
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations  # get the next state
            rewards = env_info.rewards  # get the reward
            dones = env_info.local_done  # see if episode has finished
            for an_agent in range(num_agents):
                agent.step(states[an_agent], actions[an_agent], rewards[an_agent], next_states[an_agent], dones[an_agent])
            states = next_states
            scores += rewards
            if np.any(dones):
                break
        scores_window.append(scores.mean())  # save most recent score
        avg_scores.append(scores.mean())  # save most recent score

        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if i_episode % 10 == 0: # Check every 10 episodes
            if np.mean(scores_window)>=30.0:
                torch.save(agent.actor_local.state_dict(), 'checkpoint_actor_local.pth')           # save local actor
                torch.save(agent.actor_target.state_dict(), 'checkpoint_actor_target.pth')         # save target actor
                torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_local.pth')         # save local critic
                torch.save(agent.critic_target.state_dict(), 'checkpoint_critic_target.pth')       # target critic

    print('\nEnvironment finished in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
    return avg_scores

In [ ]:
n_episodes = 300
scores = trainFunction(n_episodes)

Episode 1	Average Score: 0.78
Episode 2	Average Score: 1.81
Episode 3	Average Score: 2.74
Episode 4	Average Score: 3.84
Episode 5	Average Score: 6.67
Episode 6	Average Score: 9.69
Episode 7	Average Score: 11.77
Episode 8	Average Score: 12.44
Episode 9	Average Score: 12.66
Episode 10	Average Score: 13.90
Episode 11	Average Score: 14.89
Episode 12	Average Score: 15.50
Episode 13	Average Score: 15.91
Episode 14	Average Score: 16.49
Episode 15	Average Score: 17.36
Episode 16	Average Score: 17.92
Episode 17	Average Score: 18.54
Episode 18	Average Score: 19.07
Episode 19	Average Score: 19.74
Episode 20	Average Score: 20.11
Episode 21	Average Score: 20.56
Episode 22	Average Score: 21.06
Episode 23	Average Score: 21.70
Episode 24	Average Score: 22.11
Episode 25	Average Score: 22.43
Episode 26	Average Score: 22.70
Episode 27	Average Score: 23.03
Episode 28	Average Score: 23.45
Episode 29	Average Score: 23.82
Episode 30	Average Score: 23.73
Episode 31	Average Score: 23.85
Episode 32	Average Scor

In [ ]:
with open('results_episodes_.csv', mode='w') as save_scores:
    save_sc= csv.writer(save_scores, delimiter=',')
    for i,el in enumerate(scores):
        save_sc.writerow([i,el])

In [ ]:
import pandas as pd
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores,'b')
rolling_mean = pd.Series(scores).rolling(100).mean()
plt.plot(np.arange(len(scores)), rolling_mean,'r')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show


And now, let's observer the results of the last one!

In [ ]:
from ddpg_multi_agent import Agent
def testFunction():
    
    agent = Agent(state_size=33, action_size=4, seed=0)
    agent.actor_local.load_state_dict(torch.load('checkpoint_actor_local.pth'))
    agent.actor_target.load_state_dict(torch.load('checkpoint_actor_target.pth'))
    agent.critic_local.load_state_dict(torch.load('checkpoint_critic_local.pth'))
    agent.critic_target.load_state_dict(torch.load('checkpoint_critic_target.pth'))   
    print("Loading weights!")

    env_info = env.reset(train_mode=False)[brain_name]  # reset the environment
    states = env_info.vector_observations
    scores = np.zeros(num_agents)
    while True:
        actions =[]
        for an_agent in range(num_agents):
            agent.reset()
            actions.append(agent.act(states[an_agent]))
        
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations  # get the next state
        rewards = env_info.rewards  # get the reward
        dones = env_info.local_done  # see if episode has finished
        for an_agent in range(num_agents):
                agent.step(states[an_agent], actions[an_agent], rewards[an_agent], next_states[an_agent], dones[an_agent])
        states = next_states
        scores  +=  rewards
        if np.any(dones):
                break

    print("Score: {}".format(scores.mean()))

In [ ]:
testFunction()

In [ ]:
env.close()